# Natural Language to SQL: Fine-Tuning Llama3 with Unsloth on Google Colab

Natural Language to SQL (NL2SQL) has been a field of open research for the last years, as the integration of large-scale language models in business applications is rapidly evolving.
With the advancements of Generative AI and Large Language Models (LLMs) for code generation, the focus of the community has partially shifted from using smaller fine-tuned models for NL2SQL to larger, commercially available LLMs like GPT-4, Anthropic Claude, or Amazon Titan. Although the zero-shot capabilities of those models are impressive on benchmark datasets (e.g., SPIDER), their performance sees substantial improvements when using few-shot prompting. These results are encouraging for ML practitioners to realize the dream of using natural language for data analytics. Still, many real-world challenges remain outside the benchmarks.



In [1]:
import os
from google.colab import userdata

# if you work on Google Colab use the following to retrieve your API keys
hf_api_key = userdata.get('HUGGINGFACE_HUB')

## 2. Load and prepare the dataset
Our experiment will be based on the Spider Dataset, which is an open-source dataset for natural language to SQL, licensed under the CC BY-SA 4.0 license. In a preprocessing step, I enriched the dataset that is available on the HuggingFace hub with more information on the table schemas, foreign and primary keys for fine-tuning Llama models. [SQL-Palm](https://arxiv.org/abs/2306.00739) inspired this enrichment step and the dataset is available on the Huggingface dataset hub.

As the final model would most probably be used as a tool in a chat setting, our final dataset should reflect an instruction tuning approach, which incorporates instruction, context, and answer, as shown by an example below.

In the answer, we want Llama3 to include tags for the SQL query for easier parsing of the output.

After formatting, a training example has the following parts

### Instruction
An instruction helps the model to pick up the intent. This instruction has not been tuned. Feel free to improve it!

```
"""
### Instruction
Given an input question, use sqlite syntax to generate a sql query by choosing one or multiple of the following tables.
The foreign and primary keys will be supplied. Write query in between <SQL></SQL>.
Answer the following question with the context below:
```

### Context
The aim is to include information about the table structure via the “context” of our query. This information can be extracted from the table itself. Below is a shortened example of a relatively simple database.
```
"""
### Context
[Schema (values) (types)]: | driving_school |  Addresses : address_id (text) , line_1_number_building (number) , city (text) , zip_postcode (text) , state_province_county (text) , country (text) | Staff : staff_id (text) , staff_address_id (number) , nickname (text) , first_name (text) , middle_name (text) , last_name (text) , date_of_birth (text) , date_joined_staff (number) , date_left_staff (number) | Vehicles : vehicle_id (text) , vehicle_details (number) | Customers : customer_id (text) , customer_address_id (number) , customer_status_code (text) , date_became_customer (text) , date_of_birth (text) , first_name (text) , last_name (text) , amount_outstanding (number) , email_address (number) , phone_number (text) , cell_mobile_phone_number (text) | Customer_Payments : customer_id (text) , datetime_payment (number) , payment_method_code (text) , amount_payment (text) | Lessons : lesson_id (text) , customer_id (number) , lesson_status_code (text) , staff_id (text) , vehicle_id (text) , lesson_date (text) , lesson_time (text) , price (number); | [Foreign Keys]: staff : staff_address_id = addresses : address_id | customers : customer_address_id = addresses : address_id | customer_payments : customer_id = customers : customer_id | lessons : customer_id = customers : customer_id | lessons : staff_id = staff : staff_id | lessons : vehicle_id = vehicles : vehicle_id | [Primary Keys]: addresses : address_id, staff : staff_id, vehicles : vehicle_id, customers : customer_id, customer_payments : customer_id, lessons : lesson_id
"""
```
### Task

Experimentation with API-based and non-fine-tuned model have shown that controlling for the output — to only include the SQL query — can be a challenging task. We want our model to follow precisely our formatting rules, even when running thousands of queries. For responses in JSON format, update the instructions.
```
"""
### Answer
<SQL> SELECT T1.state_province_county FROM Addresses AS T1 JOIN Staff AS T2 ON T1.address_id = T2.staff_address_id GROUP BY T1.state_province_county HAVING count(*) BETWEEN 2 AND 4; </SQL>"""
```
To load the philikai/Spider-SQL-LLAMA2_train dataset, we use the load_dataset() method from the 🤗 Datasets library.

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("philikai/Spider-SQL-LLAMA2_train")

print(f"Train dataset size: {len(dataset)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8659 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

Train dataset size: 2


Inspecting the dataset, we find the following features:

In [4]:
dataset["train"].features


{'db_id': Value(dtype='string', id=None),
 'query': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'schema': Value(dtype='string', id=None),
 'primary_keys': Value(dtype='string', id=None),
 'foreign_keys': Value(dtype='string', id=None)}

In [5]:
dataset["train"].to_pandas().head()

,db_id,query,question,schema,primary_keys,foreign_keys
0,department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...,[Schema (values) (types)]: | department_manage...,"[Primary Keys]: department : department_id, he...",[Foreign Keys]: management : head_id = head : ...
1,department_management,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads...",[Schema (values) (types)]: | department_manage...,"[Primary Keys]: department : department_id, he...",[Foreign Keys]: management : head_id = head : ...
2,department_management,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac...",[Schema (values) (types)]: | department_manage...,"[Primary Keys]: department : department_id, he...",[Foreign Keys]: management : head_id = head : ...
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...,[Schema (values) (types)]: | department_manage...,"[Primary Keys]: department : department_id, he...",[Foreign Keys]: management : head_id = head : ...
4,department_management,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...,[Schema (values) (types)]: | department_manage...,"[Primary Keys]: department : department_id, he...",[Foreign Keys]: management : head_id = head : ...


To instruction tune our model via the prompt parts outlined above, we need to convert our structured examples into a collection of tasks described via instructions. We define a formatting_function that takes a sample and returns a string with our format instruction.

In [6]:
# Play around with the instruction prompt to maximize the model performance further
def format_spider(sample, train=True):
    instruction_prompt = f"""Given an input question, use sqlite syntax to generate a sql query by choosing one or multiple of the following tables.
    The foreign and primary keys will be supplied. Write query in between <SQL></SQL>.
    Answer the following question with the context below: \n{sample['question']}"""
    instruction = f"### Instruction\n{instruction_prompt} "
    context = f"### Context\n{sample['schema']} | {sample['foreign_keys']} | {sample['primary_keys']}"
    response = f"### Answer\n<SQL> {sample['query']} </SQL>"

    if train == True:
      # join all the parts together
      prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    else:
      prompt = "\n\n".join([i for i in [instruction, context, "### Answer\n"] if i is not None])
    return prompt

lets test our formatting function on a random example.


In [7]:
from random import randrange

print(format_spider(dataset["train"][randrange(len(dataset["train"]))]))

### Instruction
Given an input question, use sqlite syntax to generate a sql query by choosing one or multiple of the following tables.
    The foreign and primary keys will be supplied. Write query in between <SQL></SQL>.
    Answer the following question with the context below: 
Which countries have more than one mountain? 

### Context
[Schema (values) (types)]: | climbing |  mountain : mountain_id (text) , name (number) , height (text) , prominence (number) , range (number) , country (text) | climber : climber_id (text) , name (number) , country (text) , time (number) , points (number) , mountain_id (text); | [Foreign Keys]: climber : mountain_id = mountain : mountain_id | [Primary Keys]: mountain : mountain_id, climber : climber_id

### Answer
<SQL> SELECT Country FROM mountain GROUP BY Country HAVING COUNT(*)  >  1 </SQL>


### Tokenizing the dataset

Although an integral part of any LLM application, the role of the tokenizer is often overlooked. However, adding or forgetting to add a `eos_token` to your datasets samples can make the difference between a successful or failed fine-tuning job.  
You can download the right tokenizer from the Huggingface Hub. The pad_token gets set to the EOS token to ensure that the model will pick it up during the training process.


In [8]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [9]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [10]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)



config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [11]:
# assign just the train dataset for testing purposes
dataset_train = dataset["train"]
dataset_validation = dataset["validation"]

# remove 'text' column if it exists in the dataset_train
if "text" in dataset_train.column_names:
    dataset_train = dataset_train.remove_columns("text")
print(dataset_train)

# remove 'text' column if it exists in the dataset_validation
if "text" in dataset_validation.column_names:
    dataset_validation = dataset_validation.remove_columns("text")
print(dataset_validation)

Dataset({
    features: ['db_id', 'query', 'question', 'schema', 'primary_keys', 'foreign_keys'],
    num_rows: 8659
})
Dataset({
    features: ['db_id', 'query', 'question', 'schema', 'primary_keys', 'foreign_keys'],
    num_rows: 1034
})


When templating the dataset, adding an eos_token to the end of every sample, ensures that the model will stop generating after it has finished the SQL code. Furthermore, we fine-tune the model to wrap all the output code in <SQL> </SQL> tags, for us to easily parse the output.


In [12]:
from random import randint
from itertools import chain
from functools import partial


# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_spider(sample)}{tokenizer.eos_token}"
    return sample


# apply prompt template per sample
dataset_train_format_ok = dataset_train.map(
    template_dataset, remove_columns=list(dataset_train.features)
)

dataset_train_format_ok_val = dataset_validation.map(
    template_dataset, remove_columns=list(dataset_validation.features)
)
# print random sample
print(dataset_train_format_ok[randint(0, len(dataset_train_format_ok))]["text"])
print("*" * 250)
print(dataset_train_format_ok_val[randint(0, len(dataset_train_format_ok_val))]["text"])

Map:   0%|          | 0/8659 [00:00<?, ? examples/s]

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

### Instruction
Given an input question, use sqlite syntax to generate a sql query by choosing one or multiple of the following tables.
    The foreign and primary keys will be supplied. Write query in between <SQL></SQL>.
    Answer the following question with the context below: 
Retrieve the open and close dates of all the policies associated with the customer whose name contains "Diana" 

### Context
[Schema (values) (types)]: | insurance_fnol |  Customers : customer_id (text) , customer_name (number) | Services : service_id (text) , service_name (number) | Available_Policies : policy_id (text) , policy_type_code (number) , customer_phone (text) | Customers_Policies : customer_id (text) , policy_id (number) , date_opened (text) , date_closed (number) | First_Notification_of_Loss : fnol_id (text) , customer_id (number) , policy_id (text) , service_id (number) | Claims : claim_id (text) , fnol_id (number) , effective_date (text) | Settlements : settlement_id (text) , claim_id (number)

In [13]:
dataset_train_format_ok

Dataset({
    features: ['text'],
    num_rows: 8659
})

In [14]:
number_of_training_samples = 50
short_train_set = dataset_train_format_ok.select(list(range(number_of_training_samples)))

In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = short_train_set,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 3,
        num_train_epochs = 5,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

In [16]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.605 GB of memory reserved.


In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 50 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.085800
2,1.984700
3,1.776600
4,1.381700
5,0.964000
6,0.559300
7,0.569900
8,0.325600
9,0.230800
10,0.173500


In [18]:
model.save_pretrained_merged("philikai_super_duper_NL2SQL_model", tokenizer, save_method = "merged_16bit",)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.52 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:07<00:00,  2.11s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving philikai_super_duper_NL2SQL_model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving philikai_super_duper_NL2SQL_model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving philikai_super_duper_NL2SQL_model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving philikai_super_duper_NL2SQL_model/pytorch_model-00004-of-00004.bin...
Done.


In [19]:
#@title Load Custom Model
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "philikai_super_duper_NL2SQL_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Next we are going to test our model with the same prompt, but on different databases.



In [20]:
dataset_train[4000]

{'db_id': 'ship_mission',
 'query': 'SELECT Name FROM ship ORDER BY Tonnage ASC',
 'question': 'List the name of ships in ascending order of tonnage.',
 'schema': '[Schema (values) (types)]: | ship_mission |  mission : mission_id (text) , ship_id (number) , code (number) , launched_year (text) , location (number) , speed_knots (text) , fate (number) | ship : ship_id (text) , name (number) , type (number) , nationality (text) , tonnage (number);',
 'primary_keys': '[Primary Keys]: mission : mission_id, ship : ship_id',
 'foreign_keys': '[Foreign Keys]: mission : ship_id = ship : ship_id'}

In [21]:
query = format_spider(dataset_train[4000], train=False)

In [22]:
#### Predict
inputs = tokenizer([query], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.001)
model_prediction = tokenizer.batch_decode(outputs)[0]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [23]:
from pprint import pprint
pprint(model_prediction)

('<|begin_of_text|>### Instruction\n'
 'Given an input question, use sqlite syntax to generate a sql query by '
 'choosing one or multiple of the following tables.\n'
 '    The foreign and primary keys will be supplied. Write query in between '
 '<SQL></SQL>.\n'
 '    Answer the following question with the context below: \n'
 'List the name of ships in ascending order of tonnage. \n'
 '\n'
 '### Context\n'
 '[Schema (values) (types)]: | ship_mission |  mission : mission_id (text), '
 'ship_id (number), code (number), launched_year (text), location (number), '
 'speed_knots (text), fate (number) | ship : ship_id (text), name (number), '
 'type (number), nationality (text), tonnage (number); | [Foreign Keys]: '
 'mission : ship_id = ship : ship_id | [Primary Keys]: mission : mission_id, '
 'ship : ship_id\n'
 '\n'
 '### Answer\n'
 '<SQL>SELECT name FROM ship ORDER BY tonnage ASC;</SQL><|end_of_text|>')


In [24]:
pprint(dataset_train[4000]["question"])
pprint(dataset_train[4000]["query"])

'List the name of ships in ascending order of tonnage.'
'SELECT Name FROM ship ORDER BY Tonnage ASC'


## Testing our new model with some slightly out of sample synthetic data!


Next we are going to load some synthetic data that was generated with Claude 3 Haiku.

The data was generated with the follwing prompt:
```SQL_flavor = "sqlite"

SystemPrompt = f"You are a Professor teaching databases at for informatics students. Your task is to setup a new questions for an upcoming quiz/examination based on {SQL_flavor}."

PromptTemplate = """
Below is a database schema and {n} already existing questions on that database schema that are created by experts that are called "Ground Truth Exam Questions and Answers".\n
Then there are the newly created Exam Questions and Answers, which where created by the AI model.
Those question and answer pairs are for inspiration only. Never create a questions that is the same as the "Ground Truth Exam Questions and Answers"! \n

---------------------
Database Schema:
{DDL_schema}

Existing Ground Truth Exam Questions and Answers:
{GroundTruth_QnAPairs}

Created New  Exam Questions and Answers:
{AI_QnAPairs}
---------------------

Given the context information above generate only questions based on the below query.

The questions should be diverse in nature and difficulty \
across the database. You can create more difficult questions than shown above.  The questions should not contain options, not start with Q1/ Q2. \
Output only the question as well as the SQL query that answers the questions pairs between <NewPair></NewPair> tags!\
Then add extra tags for the question and the answer and the complexity. \n
The exam question have 3 different complexity metrics based on the complexity of their queries. Here is the complexity mapping:\n
Aggregate Functions - Complexity: 1\n
Data Filtering and Sorting (WHERE and ORDER BY clauses) - Complexity: 1\n
SQL String Functions and Date Functions - Complexity: 1\n
Views - Complexity: 2\n
Group By and Having Clauses - Complexity: 2\n
Joins - Complexity: 2\n
Indexes - Complexity: 2\n
CASE Statements - Complexity: 2\n
Subqueries and Common Table Expressions (CTEs) - Complexity: 3\n
Window Functions - Complexity: 3\n
Transactions - Complexity: 3\n
If the complexity of the database schema allows, try to create complexity 2 and 3 questions. \

An example of a question and answer pair that format is shown below:\n
    <NewPair>
    <question>What is the average age of singers performing in concerts held at
    the stadium with the highest average attendance?</question>
    
    <answer>
    SELECT AVG(T2.age)
    FROM singer AS T2 JOIN singer_in_concert AS T1 ON T2.singer_id =
    T1.singer_id
    JOIN concert AS T3 ON T1.concert_id = T3.concert_id
    JOIN stadium AS T4 ON T3.stadium_id = T4.stadium_id
    WHERE T4.stadium_id = (SELECT stadium_id FROM stadium ORDER BY average DESC
    LIMIT 1)
    </answer>
    <complexity>2</complexity>

    </NewPair>

Create {n_new} new questions and answers based on the above query. Do not repeat the same questions.

\n\nAssistant:\"""
"""
```

In [25]:
# Load dataset from the hub
dataset_inference = load_dataset("philikai/SPIDER_SQL_synth_data_w_Claude3_Haiku")

dataset_inference_train = dataset_inference["train"]

dataset_inference_train.to_pandas().head()

Generating train split:   0%|          | 0/10257 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5051 [00:00<?, ? examples/s]

,db_id,DDL_schema,question,query,complexity,execution_result,error_flag,exception,__index_level_0__
0,department_management,"CREATE TABLE ""department"" (\n""Department_ID"" i...",What is the total budget of the departments th...,SELECT SUM(budget_in_billions) AS total_budget...,1,"[(743.6000000000001,)]",False,None,0
1,department_management,"CREATE TABLE ""department"" (\n""Department_ID"" i...",List the names of the departments that were cr...,SELECT name FROM department WHERE strftime('%Y...,1,"[('Homeland Security',), ('Veterans Affairs',)...",False,None,1
2,department_management,"CREATE TABLE ""department"" (\n""Department_ID"" i...","Create a view that shows the department name, ...",CREATE VIEW department_head_view AS SELECT d.n...,2,[],False,None,2
3,department_management,"CREATE TABLE ""department"" (\n""Department_ID"" i...",Which departments have a budget greater than t...,"SELECT d.name, d.budget_in_billions, d.num_emp...",2,"[('Defense', 439.3, 3000000.0), ('Health and H...",False,None,3
4,department_management,"CREATE TABLE ""department"" (\n""Department_ID"" i...","List the department names, head names, and the...","SELECT d.name AS department_name, h.name AS he...",3,[],False,None,4


In [26]:
# Format a sammple
# Play around with the instruction prompt to maximize the model performance further
def format_spider_DDL(sample, train=True, include_context = True):
    instruction_prompt = f"""Given an input question, use sqlite syntax to generate a sql query by choosing one or multiple of the following tables.
    The DDL of the database will be supplied. Write query in between <SQL></SQL>.
    Answer the following question with the context below: \n{sample['question']}"""
    instruction = f"### Instruction\n{instruction_prompt} "
    context = f"### Context\n{sample['DDL_schema']}"
    response = f"### Answer\n<SQL> {sample['query']} </SQL>"

    if include_context == False:
      context = ""

    if train == True:
      # join all the parts together
      prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    else:
      prompt = "\n\n".join([i for i in [instruction, context, "### Answer\n"] if i is not None])
    return prompt


In [32]:
synth_sample = dataset_inference_train[0]
pprint(format_spider_DDL(synth_sample))

('### Instruction\n'
 'Given an input question, use sqlite syntax to generate a sql query by '
 'choosing one or multiple of the following tables.\n'
 '    The DDL of the database will be supplied. Write query in between '
 '<SQL></SQL>.\n'
 '    Answer the following question with the context below: \n'
 'What is the total budget of the departments that have a ranking between 5 '
 'and 10? \n'
 '\n'
 '### Context\n'
 'CREATE TABLE "department" (\n'
 '"Department_ID" int,\n'
 '"Name" text,\n'
 '"Creation" text,\n'
 '"Ranking" int,\n'
 '"Budget_in_Billions" real,\n'
 '"Num_Employees" real,\n'
 'PRIMARY KEY ("Department_ID")\n'
 ')\n'
 '\n'
 'CREATE TABLE "head" (\n'
 '"head_ID" int,\n'
 '"name" text,\n'
 '"born_state" text,\n'
 '"age" real,\n'
 'PRIMARY KEY ("head_ID")\n'
 ')\n'
 '\n'
 'CREATE TABLE "management" (\n'
 '"department_ID" int,\n'
 '"head_ID" int,\n'
 '"temporary_acting" text,\n'
 'PRIMARY KEY ("Department_ID","head_ID"),\n'
 'FOREIGN KEY ("Department_ID") REFERENCES `departm

In [33]:
#### Predict on synthetic data
inputs = tokenizer([format_spider_DDL(synth_sample, train=False)], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 150, temperature=0.001)
synth_test_answer = tokenizer.batch_decode(outputs)[0]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [34]:
pprint(synth_test_answer)

('<|begin_of_text|>### Instruction\n'
 'Given an input question, use sqlite syntax to generate a sql query by '
 'choosing one or multiple of the following tables.\n'
 '    The DDL of the database will be supplied. Write query in between '
 '<SQL></SQL>.\n'
 '    Answer the following question with the context below: \n'
 'What is the total budget of the departments that have a ranking between 5 '
 'and 10? \n'
 '\n'
 '### Context\n'
 'CREATE TABLE "department" (\n'
 '"Department_ID" int,\n'
 '"Name" text,\n'
 '"Creation" text,\n'
 '"Ranking" int,\n'
 '"Budget_in_Billions" real,\n'
 '"Num_Employees" real,\n'
 'PRIMARY KEY ("Department_ID")\n'
 ')\n'
 '\n'
 'CREATE TABLE "head" (\n'
 '"head_ID" int,\n'
 '"name" text,\n'
 '"born_state" text,\n'
 '"age" real,\n'
 'PRIMARY KEY ("head_ID")\n'
 ')\n'
 '\n'
 'CREATE TABLE "management" (\n'
 '"department_ID" int,\n'
 '"head_ID" int,\n'
 '"temporary_acting" text,\n'
 'PRIMARY KEY ("Department_ID","head_ID"),\n'
 'FOREIGN KEY ("Department_ID") RE

In [35]:
#@title removing the context
format_spider_DDL(synth_sample, train=False, include_context = False)

'### Instruction\nGiven an input question, use sqlite syntax to generate a sql query by choosing one or multiple of the following tables.\n    The DDL of the database will be supplied. Write query in between <SQL></SQL>.\n    Answer the following question with the context below: \nWhat is the total budget of the departments that have a ranking between 5 and 10? \n\n\n\n### Answer\n'

In [36]:
#### Predict on synthetic data
inputs = tokenizer([format_spider_DDL(synth_sample, train=False, include_context = False)], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 150, temperature=0.001)
synth_test_answer_no_context = tokenizer.batch_decode(outputs)[0]
pprint(synth_test_answer_no_context)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


('<|begin_of_text|>### Instruction\n'
 'Given an input question, use sqlite syntax to generate a sql query by '
 'choosing one or multiple of the following tables.\n'
 '    The DDL of the database will be supplied. Write query in between '
 '<SQL></SQL>.\n'
 '    Answer the following question with the context below: \n'
 'What is the total budget of the departments that have a ranking between 5 '
 'and 10? \n'
 '\n'
 '\n'
 '\n'
 '### Answer\n'
 '```sql\n'
 'SELECT SUM(budget) FROM departments WHERE ranking BETWEEN 5 AND 10;\n'
 '```\n'
 '<|end_of_text|>')


## Results (exemplary)

A sample result, by running the model on a database it has not been trained on:
```
 '### Answer\n'
 '<SQL> SELECT name FROM ship ORDER BY tonnage ASC </SQL>\n'
 '\n'
 '### Explanation\n'
 'The query lists the name of ships in ascending order of tonnage. \n'
 '<|end_of_text|>
```

A sample result, by running the model on a database it has been trained on, but with different looking instruction:
```
'### Answer\n'
 '<SQL>\n'
 'SELECT SUM(Budget_in_Billions) FROM department WHERE Ranking BETWEEN 5 AND '
 '10;\n'
 '</SQL><|end_of_text|>'
```

By carefully comparing the outputs, we can observe, that even though we are out of sample (OOS) on the instruction set - SQL-PALM format for training vs DDL format for inference - the model recognizes that it has to output only the SQL in the ```<SQL>``` tags.

Even when we remove the context from the query, we can still get the output of the model to be in-line with our expectations, however, it made a mistake to sum `budget` and not `Budget_in_Billions`:
```
 '### Answer\n'
 '```sql\n'
 'SELECT SUM(budget) FROM departments WHERE ranking BETWEEN 5 AND 10;\n'
 '```\n'
 '<|end_of_text|>'
 ```

You are encouraged to test you model further!

Please note that your output can be different due to training.
